In [1]:
import os
import pyspark
from pyspark.sql import SparkSession
memory = '2g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

# we can verify the memory in the spark server/Environment

# An alternative mechanism for setting memory is 
# https://docs.qubole.com/en/latest/user-guide/notebooks-and-dashboards/notebooks/jupyter-notebooks/config-spark-settings-jupy.html

In [1]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2022-02-25 21:15:09--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.139.0
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.139.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv.1’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2022-02-25 21:15:10 (91.1 MB/s) - ‘taxi+_zone_lookup.csv.1’ saved [12322/12322]



In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

df.show()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/25 21:30:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [3]:
sc = SparkContext(conf=conf)
sc._conf.get('spark.driver.memory')

NameError: name 'sc' is not defined

In [ ]:
!wc -l taxi+_zone_lookup.csv

In [ ]:
df.write.parquet('zones')

In [ ]:
!ls

In [ ]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

In [ ]:
!wc -l fhvhv_tripdata_2021-01.csv

In [ ]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

df.show()

In [ ]:
spark.version

In [ ]:
df.head(5)

In [ ]:
type(df.head(1)[0])

In [ ]:
df.schema

In [ ]:
!head -n 101 fhvhv_tripdata_2021-01.csv > head.csv

In [ ]:
!wc -l head.csv

In [ ]:
import pandas as pd
df_pd = pd.read_csv('head.csv')
df_pd.dtypes

In [ ]:
spark.createDataFrame(df_pd).show()

In [ ]:
spark.createDataFrame(df_pd).schema

In [4]:
from pyspark.sql import types
schema = types.StructType([
	types.StructField('hvfhs_license_num',types.StringType(),True),
	types.StructField('dispatching_base_num',types.StringType(),True),
	types.StructField('pickup_datetime',types.TimestampType(),True),
	types.StructField('dropoff_datetime',types.TimestampType(),True),
	types.StructField('PULocationID',types.IntegerType(),True),
	types.StructField('DOLocationID',types.IntegerType(),True),
	types.StructField('SR_Flag',types.StringType(),True)
])

In [5]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [ ]:
df.schema

In [ ]:
df.head(5)

In [9]:
# This is important to update the actual dataframe
df = df.repartition(24)

In [10]:
df.write.parquet('fhvhv/2021/01/') # use mode='overwrite'

22/02/25 21:35:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 96.51% for 14 writers
22/02/25 21:35:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 90.08% for 15 writers
22/02/25 21:35:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 84.45% for 16 writers
22/02/25 21:35:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 90.08% for 15 writers
22/02/25 21:35:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 96.51% for 14 writers


In [ ]:
!ls

In [12]:
!ls -lh fhvhv/2021/01 | wc -l

      26


### More about Dataframes

In [13]:
df = spark.read.parquet('fhvhv/2021/01/')

In [16]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [15]:
df.head(1)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02870', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 32, 59), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 50, 47), PULocationID=169, DOLocationID=24, SR_Flag=None)]

In [17]:
df.select(['hvfhs_license_num','dropoff_datetime'])

DataFrame[hvfhs_license_num: string, dropoff_datetime: timestamp]

In [18]:
df.select('hvfhs_license_num','dropoff_datetime')

DataFrame[hvfhs_license_num: string, dropoff_datetime: timestamp]

In [23]:
df.select('hvfhs_license_num','pickup_datetime','dropoff_datetime') \
.filter(df.hvfhs_license_num == 'HV0003').show()

+-----------------+-------------------+-------------------+
|hvfhs_license_num|    pickup_datetime|   dropoff_datetime|
+-----------------+-------------------+-------------------+
|           HV0003|2021-01-01 00:32:59|2021-01-01 00:50:47|
|           HV0003|2021-01-01 18:26:07|2021-01-01 18:43:41|
|           HV0003|2021-01-02 20:44:31|2021-01-02 20:55:49|
|           HV0003|2021-01-02 21:36:26|2021-01-02 21:56:20|
|           HV0003|2021-01-02 19:29:32|2021-01-02 19:42:29|
|           HV0003|2021-01-02 16:13:07|2021-01-02 16:24:15|
|           HV0003|2021-01-02 00:21:47|2021-01-02 00:49:22|
|           HV0003|2021-01-01 13:58:53|2021-01-01 14:04:00|
|           HV0003|2021-01-01 04:44:33|2021-01-01 04:51:26|
|           HV0003|2021-01-01 14:44:06|2021-01-01 14:55:28|
|           HV0003|2021-01-01 21:08:01|2021-01-01 21:17:55|
|           HV0003|2021-01-02 14:34:15|2021-01-02 14:51:51|
|           HV0003|2021-01-01 02:04:44|2021-01-01 02:25:29|
|           HV0003|2021-01-02 21:22:46|2

![Actions_vs_Transformers](actions_transformers.png)

`SELECT * from df where hvfhs_license_num= HV003`

In [25]:
from pyspark.sql import functions as F

In [44]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select(['pickup_date','dropoff_date','PULocationID','DOLocationID']) \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-01|  2021-01-01|         169|          24|
| 2021-01-01|  2021-01-01|         141|         231|
| 2021-01-02|  2021-01-02|         197|          10|
| 2021-01-02|  2021-01-02|         146|         244|
| 2021-01-02|  2021-01-02|         197|         215|
| 2021-01-02|  2021-01-02|         161|         233|
| 2021-01-02|  2021-01-02|         227|          89|
| 2021-01-01|  2021-01-01|          95|          95|
| 2021-01-01|  2021-01-01|         248|         235|
| 2021-01-01|  2021-01-01|           4|         113|
| 2021-01-01|  2021-01-01|          47|         126|
| 2021-01-02|  2021-01-02|           7|         263|
| 2021-01-01|  2021-01-01|          48|          74|
| 2021-01-02|  2021-01-02|         238|         100|
| 2021-01-02|  2021-01-02|         202|         138|
| 2021-01-02|  2021-01-02|          61|       

In [47]:
# Custom function
def crazy_stuff(base_num):
    num = int(base_num[1:])
    return f's/{num:03x}' if num%7==0 else f'e/{num:03x}' 

In [48]:
crazy_stuff('B02884')

's/b44'

In [51]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [53]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select(['pickup_date','base_id','dropoff_date','PULocationID','DOLocationID']) \
    .show()

+-----------+-------+------------+------------+------------+
|pickup_date|base_id|dropoff_date|PULocationID|DOLocationID|
+-----------+-------+------------+------------+------------+
| 2021-01-01|  s/b36|  2021-01-01|         169|          24|
| 2021-01-01|  e/b38|  2021-01-01|         141|         231|
| 2021-01-02|  e/acc|  2021-01-02|         197|          10|
| 2021-01-02|  s/acd|  2021-01-02|         146|         244|
| 2021-01-02|  e/a39|  2021-01-02|         197|         215|
| 2021-01-02|  e/b14|  2021-01-02|         161|         233|
| 2021-01-02|  e/acc|  2021-01-02|         227|          89|
| 2021-01-01|  s/b3d|  2021-01-01|          95|          95|
| 2021-01-01|  e/b47|  2021-01-01|         248|         235|
| 2021-01-01|  e/b49|  2021-01-01|           4|         113|
| 2021-01-01|  e/95b|  2021-01-01|          47|         126|
| 2021-01-02|  e/a39|  2021-01-02|           7|         263|
| 2021-01-01|  e/b37|  2021-01-01|          48|          74|
| 2021-01-02|  e/b35|  2